#  Importing the Data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/cleaned_dataset (1).csv')

In [ ]:
X = df[['Ax', 'Ay', 'Az', 'Gx', 'Gy', 'Gz']]  # Features
y = df['Gesture']  # Target (ensure it's encoded beforehand)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [ ]:
y_train.head()

,Gesture
22,0
7,0
29,0
131,1
209,1


In [ ]:
X_train.head()

,Ax,Ay,Az,Gx,Gy,Gz
22,0.431675,0.982270,0.946569,0.960115,0.193210,0.551804
7,0.368270,0.979560,0.978186,0.968941,0.285519,0.550467
29,0.266192,0.964313,0.000000,0.968225,0.281851,0.551783
131,0.472817,0.951918,0.990261,0.969358,0.250177,0.553126
209,0.328845,0.820256,0.978042,0.969214,0.269406,0.553045


# Import Dependencies

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
import tensorflow as tf


# Build,Compile and fit the model

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

# Scaling features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Model definition
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),  # Increased dropout to reduce overfitting
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Increased dropout
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),  # Batch normalization to stabilize learning
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)  # Added restore_best_weights
lr_schedule = LearningRateScheduler(lambda epoch: 1e-3 * 0.9 ** epoch)  # Learning rate decay

# Train the model
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_val, y_val),
                    callbacks=[early_stopping, lr_schedule])

# Evaluate model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Predict and Evaluate

In [ ]:
y_hat = model.predict(X_test)
y_hat = [0 if val < 0.5 else 1 for val in y_hat]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


In [ ]:
print(y_train.value_counts())


Gesture
1    70
0    66
Name: count, dtype: int64


In [ ]:
accuracy_score(y_test, y_hat)

NameError: name 'accuracy_score' is not defined

# ploting,Saving and Reloading

In [ ]:
model.save('tfmodel.keras')


In [ ]:
del model

In [ ]:
model = load_model('tfmodel.keras')

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8602 - loss: 0.3496 
Test Loss: 0.3779
Test Accuracy: 83.72%


In [ ]:
import tensorflow as tf

# Load the trained Keras model
model = tf.keras.models.load_model('/content/tf 83.keras')

# 1. Convert the trained model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# 2. Apply quantization (optional)
def representative_dataset_gen():
    for data in X_train[:100]:  # Use a subset of your training data
        # Ensure the data is of type float32
        yield [data.astype('float32')]

converter.representative_dataset = representative_dataset_gen
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Apply post-training quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  # Use INT8 quantization

# Convert the model to TFLite format
tflite_model = converter.convert()

# Save the converted model as a .tflite file
with open('quantized_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model successfully converted to TFLite format and saved as 'quantized_model.tflite'.")


Saved artifact at '/tmp/tmpep3pwr4o'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 6), dtype=tf.float32, name='input_layer_14')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  135548121047536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135548118454992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135548577706256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135548577703088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135548577706080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135548118930832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135548118931888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135548118932768: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model successfully converted to TFLite format and saved as 'quantized_model.tflite'.


/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [ ]:
y_pred = model.predict(X_test)  # Get predicted probabilities
y_pred_classes = (y_pred > 0.5).astype(int)  # Convert probabilities to class labels (0 or 1)

# Compare predictions to actual labels
print(f"Predicted classes: {y_pred_classes[:10]}")
print(f"Actual classes: {y_test[:10]}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted classes: [[0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]]
Actual classes: 93     0
192    1
201    1
51     0
56     0
133    1
33     0
67     0
200    1
4      0
Name: Gesture, dtype: int64


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)
print(f"Confusion Matrix:\n{cm}")

# Generate a classification report
cr = classification_report(y_test, y_pred_classes)
print(f"Classification Report:\n{cr}")


Confusion Matrix:
[[19  4]
 [ 3 17]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.83      0.84        23
           1       0.81      0.85      0.83        20

    accuracy                           0.84        43
   macro avg       0.84      0.84      0.84        43
weighted avg       0.84      0.84      0.84        43



In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a history object from model.fit()
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_val, y_val),
                    callbacks=[early_stopping, lr_schedule])

# Plot training & validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


NameError: name 'model' is not defined